In [1]:
import pandas as pd
import numpy as np
import datetime
import tensorflow as tf
import numpy as np
import metrics
import time
import model
from useful_fns import *
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import os

In [2]:
def plot(df):
    l = df['True'].shape[0]
    x = np.linspace(0,l-1, l)

    z_90 = 1.645 
    z_50 = 0.674
    plt.plot(x, df['True'], color='black')
    plt.plot(x, df['Pred'], color='red')
    plt.fill_between(x, df['Pred']-z_90*df['Std'],
                    df['Pred']+z_90*df['Std'], color='pink',alpha=0.5)
    plt.fill_between(x, df['Pred']-z_50*df['Std'],
                    df['Pred']+z_50*df['Std'], color='pink',alpha=0.5)

    plt.show()

In [8]:
import importlib
import model
importlib.reload(model)
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
save_dir = 'results/hyper_parameter_optimisation/res_3/'

def run_model(Batch_Size, Epochs, lr, kl_anneal, num_layers, prior_scale, rho_op, rho_q, scheduler_exp_val, scheduler_start_epoch, sizeof_bnn, sizeof_layers):
    os.chdir('/home/mimorris/Projects/Neural-Network-Flu-Uncertainty/src')
    
    args = Namespace(Arch='FF', 
                 num_layers=int(num_layers),
                 sizeof_layers=int(sizeof_layers),
                 batch_norm=True,
                 Ext='-c', 
                 kl_anneal=float(kl_anneal),
                 rho_q=float(rho_q),
                 rho_op=float(rho_op),
                 prior_scale=float(prior_scale),
                 sizeof_bnn=int(sizeof_bnn),
                 Gamma=14, 
                 country='us', 
                 early_stopping=False, 
                 smooth=True,
                 scheduler_start_epoch=int(scheduler_start_epoch), 
                 lr=lr,
                 scheduler_exp_val=scheduler_exp_val,
                 Batch_Size=int(Batch_Size), 
                 Epochs=int(Epochs), 
                     
                )
    
    results = results_table(args)
    data = data_builder_new(root="/home/mimorris/Datasets/Flu/", 
                            country = args.country)

    for year in range(1):
        try:
            tf.keras.backend.clear_session()

            x_train, y_train, x_test, y_test, y_test_full = data.build_2(test_season=2013+year, 
                                                            gamma=args.Gamma, 
                                                            lag=14, 
                                                            window_size=28, 
                                                            smooth=args.smooth)

            NN = model.model_builder(x_train, y_train.values[:, np.newaxis], args=args)
            NN.fit(x_train, y_train.values[:, np.newaxis], verbose=False)

            test_pred = NN.predict(x_test, pd.DataFrame(index=y_test_full.index, data=y_test_full.values, columns=['T0']))
            test_pred = rescale(test_pred, data)

            nt = test_pred.index == np.inf
            for j in y_test.index:
                nt = nt + (test_pred.index == j)
            test_pred = test_pred[nt]
            test_pred['True'] = y_test['ILI']

            results.update(test_pred, year=year)

            results.save(save_dir, get_num(save_dir))
        except:
            return -100
    if np.isfinite(results.test_metrics['Average']['MB Log']):
        return(np.exp(results.test_metrics['Average']['MB Log']))
    else:
        return -100

In [9]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs


pbounds = {'Batch_Size':(4, 128), 
           'Epochs':(50,500), 
           'kl_anneal': (0.1, 5.0),
           'lr':(0.00001, 0.01), 
           'num_layers': (1, 4), 
           'prior_scale':(0.01, 10), 
           'rho_op':(0.1, 10),
           'rho_q':(0.1, 10),
           'scheduler_exp_val':(-0.1, -0.00001), 
           'scheduler_start_epoch':(1,500),
           'sizeof_bnn':(2, 128),
           'sizeof_layers': (16, 512)}

           
optimizer = BayesianOptimization(
    f=run_model,
    pbounds=pbounds,
    random_state=25,
    verbose=2
)          

args = Namespace(Arch='FF', 
             num_layers=int(3),
             sizeof_layers=int(207),
             batch_norm=True,
             Ext='-c', 
             kl_anneal=float(0.4),
             rho_q=float(3.498),
             rho_op=float(1.461),
             prior_scale=float(2.32),
             sizeof_bnn=int(35),
             Gamma=14, 
             country='us', 
             early_stopping=False, 
             smooth=True,
             scheduler_start_epoch=int(12.59), 
             lr=0.003,
             scheduler_exp_val=-0.08907,
             Batch_Size=int(50), 
             Epochs=int(450), 
            )

In [10]:
optimizer.maximize(
    init_points=10,
    n_iter=10000,
)

|   iter    |  target   | Batch_... |  Epochs   | kl_anneal |    lr     | num_la... | prior_... |  rho_op   |   rho_q   | schedu... | schedu... | sizeof... | sizeof... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -100.0    |  111.9    |  312.0    |  1.466    |  0.001867 |  2.233    |  1.183    |  6.881    |  4.432    | -0.04438  |  184.2    |  52.7     |  72.07    |
|  2        |  0.05933  |  59.43    |  313.5    |  0.8937   |  0.005212 |  1.978    |  6.995    |  3.727    |  8.38     | -0.05187  |  258.7    |  50.26    |  510.8    |
|  3        |  0.05616  |  67.77    |  301.6    |  0.2688   |  0.007202 |  2.263    |  4.375    |  2.889    |  9.013    | -0.03305  |  228.6    |  38.52    |  276.8    |
|  4        | -100.0    |  73.35    |  385.4    |  4.159    |  0.008239 |  1.231    |  6.452    |  3.162    |  5.29     | -0.0042   |  441.7    |  39.

/home/mimorris/Projects/Neural-Network-Flu-Uncertainty/src/metrics.py:131: RuntimeWarning: divide by zero encountered in log
  return np.log((dist.cdf(true+0.6) - dist.cdf(true-0.5)).numpy())


|  6        | -100.0    |  53.17    |  421.3    |  3.592    |  0.000988 |  3.526    |  4.414    |  3.802    |  9.145    | -0.04523  |  126.7    |  5.462    |  118.3    |
|  7        | -100.0    |  77.27    |  123.6    |  4.201    |  0.007754 |  1.507    |  7.672    |  3.42     |  4.777    | -0.0785   |  456.1    |  97.66    |  351.6    |
|  8        |  0.3514   |  6.651    |  347.4    |  0.5628   |  0.008313 |  1.338    |  5.673    |  1.829    |  7.927    | -0.09663  |  398.2    |  88.87    |  260.0    |
|  9        |  0.2543   |  14.98    |  471.9    |  0.5134   |  0.004699 |  3.417    |  0.8656   |  2.519    |  8.939    | -0.05214  |  96.01    |  33.88    |  394.1    |
|  10       | -100.0    |  122.9    |  393.0    |  1.668    |  0.007918 |  2.469    |  2.971    |  6.059    |  2.297    | -0.0614   |  4.292    |  47.13    |  113.6    |
|  11       | -100.0    |  75.45    |  289.2    |  0.5443   |  0.005257 |  2.819    |  4.977    |  7.199    |  1.095    | -0.07447  |  95.39    |  73.

/home/mimorris/Projects/Neural-Network-Flu-Uncertainty/src/metrics.py:131: RuntimeWarning: divide by zero encountered in log
  return np.log((dist.cdf(true+0.6) - dist.cdf(true-0.5)).numpy())


|  16       | -100.0    |  59.53    |  252.0    |  0.1324   |  0.00528  |  2.696    |  5.047    |  1.407    |  10.0     | -0.02482  |  221.2    |  17.61    |  248.1    |
|  17       |  0.03079  |  76.34    |  328.4    |  0.3146   |  0.008407 |  2.078    |  4.061    |  3.481    |  8.717    | -0.04016  |  228.1    |  47.03    |  298.1    |


...

/home/mimorris/Projects/Neural-Network-Flu-Uncertainty/src/metrics.py:131: RuntimeWarning: divide by zero encountered in log
  return np.log((dist.cdf(true+0.6) - dist.cdf(true-0.5)).numpy())


|  459      | -100.0    |  38.76    |  257.0    |  1.813    |  0.007663 |  3.161    |  0.08175  |  2.116    |  9.762    | -0.006    |  479.7    |  6.708    |  234.6    |
|  460      | -100.0    |  76.08    |  327.1    |  0.1755   |  0.005249 |  2.202    |  8.895    |  9.359    |  9.417    | -0.09446  |  247.9    |  32.65    |  285.0    |
|  461      |  0.2552   |  9.957    |  70.55    |  4.533    |  0.002774 |  1.576    |  8.917    |  4.172    |  9.222    | -0.006331 |  293.6    |  20.23    |  277.9    |
|  462      | -100.0    |  63.46    |  330.3    |  2.427    |  0.005786 |  2.604    |  5.805    |  2.518    |  5.483    | -0.05595  |  265.3    |  60.88    |  508.8    |
|  463      | -100.0    |  86.92    |  387.4    |  1.416    |  0.001015 |  2.593    |  4.963    |  5.917    |  3.102    | -0.02519  |  316.6    |  126.3    |  374.7    |
|  464      | -100.0    |  12.43    |  365.1    |  1.08     |  0.000716 |  2.547    |  4.27     |  5.424    |  4.572    | -0.0243   |  123.4    |  111

/home/mimorris/Projects/Neural-Network-Flu-Uncertainty/src/metrics.py:131: RuntimeWarning: divide by zero encountered in log
  return np.log((dist.cdf(true+0.6) - dist.cdf(true-0.5)).numpy())


|  466      | -100.0    |  28.02    |  259.7    |  0.1      |  0.007161 |  4.0      |  10.0     |  0.6991   |  8.309    | -0.04925  |  242.3    |  47.7     |  318.9    |
|  467      |  0.2956   |  5.229    |  134.6    |  1.75     |  0.00593  |  1.046    |  6.552    |  1.004    |  7.457    | -0.08003  |  215.7    |  26.35    |  462.4    |
|  468      | -100.0    |  12.11    |  256.7    |  2.203    |  0.003335 |  3.366    |  3.449    |  9.212    |  3.244    | -0.04312  |  170.8    |  39.95    |  25.19    |
|  469      |  0.3017   |  12.49    |  103.1    |  0.2761   |  0.009959 |  1.252    |  6.481    |  1.172    |  5.793    | -0.03765  |  66.24    |  27.17    |  168.7    |
|  470      | -100.0    |  97.86    |  76.03    |  3.771    |  0.007466 |  3.644    |  7.29     |  9.33     |  5.445    | -0.01378  |  48.27    |  49.92    |  24.16    |
|  471      | -100.0    |  46.04    |  74.39    |  1.887    |  0.009449 |  2.719    |  4.997    |  7.727    |  5.158    | -0.06697  |  428.1    |  87.

KeyboardInterrupt: 

In [14]:
optimizer.max

{'target': 0.5072699259573482,
 'params': {'Batch_Size': 50.36985154384715,
  'Epochs': 456.63441396602803,
  'kl_anneal': 0.3976259246018009,
  'lr': 0.0030028125615625457,
  'num_layers': 3.7897690631642176,
  'prior_scale': 2.3198826772607224,
  'rho_op': 1.460744187779316,
  'rho_q': 3.4984128837797654,
  'scheduler_exp_val': -0.08906723352286981,
  'scheduler_start_epoch': 12.592398650899684,
  'sizeof_bnn': 35.6230050075188,
  'sizeof_layers': 207.17023236767915}}

In [ ]:
np.exp()


In [ ]:
mb_log(results.val_predictions).mean()

In [ ]:
import copy
from metrics import *

for i in range(1,15):
    temp = copy.copy(results.val_predictions)
    temp['Std']=temp['Std']*i/10
    print(i/10, mb_log(temp).mean())
    
    plot(temp)

In [ ]:
int(41.9)